IA COM PYTHON E LANGCHAIN

O que é LangChain: É um framework/ecossistema de código aberto para desenvolvimento de aplicações com grandes modelos de linguagens (LLMs). Disponível em bibliotecas Python e Java, as ferramentas e APIs do LangChain simplificam o processo de criação de aplicativos baseados em LLMs

O Django da Inteligência Artificial ! 

O ecossistema do LangChain está em acelerado crescimento e já conta com diversos  [Pacotes](https://python.langchain.com/docs/integrations/providers/):

- langchain e langchain_core
- langchain_community
- langchain_openai
- langchain_aws
- langchain_chroma

Se for usar o LLM da Openai tem que criar uma conta nesse [site](https://platform.openai.com/settings/organization/billing/overview) e após isso criar uma API para ser consumida dentro do LangChain, depois devemos adicionar essa API em um arquivo .env

Precisamos instalar alguns pacotes no terminal: 

- pip install langchain 
- pip install langchain-openai
- pip install python-decouple
- pipi isntall langchain-community

Agora vamos conectar o LangChain as modelos de linguagem:

OPENAI:

In [ ]:
import os
from decouple import config

from langchain_openai import OpenAI

os.environ['OPENAI_API_KEY'] = config('OPENAI_API_KEY')

model = OpenAI()

question = input('O que deseja saber ?')

response = model.invoke(
    input=question,
)

print(response)

O LangChain_openai possui um pacote (lib) de CHAT e com isso conseguimos criar um chat separando mensagens de um usuário com mensagens da IA:

In [ ]:
import os
from decouple import config

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage # tipos
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = config('OPENAI_API_KEY')

model = ChatOpenAI(
    model='gpt-3.5-turbo',

)

question = input('O que deseja saber ?')

message = [
    SystemMessage(content='Você é um assistente que fornece informações sobre figuras históricas.'), # definição do padrão da IA
    HumanMessage(content=question), # mensagem do usuário.
]

response = model.invoke(message)

print(response)
print(response.content)





O LangChain permite a criação de templates de prompts para usar no código:

In [ ]:
import os
from decouple import config

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage # tipos
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate # templates de menssagens


os.environ['OPENAI_API_KEY'] = config('OPENAI_API_KEY')

model = ChatOpenAI(
    model='gpt-3.5-turbo',

)

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='Você deve responder baseado em dados geográficos de regiões do Brasil e seja o mais breve possível.'),
        
        HumanMessagePromptTemplate.from_template(
            template='Por favor, me fale sobre a região {regiao}, com dados demográficos.'
        ),
    ]
)

regiao = input('sobre qual região deseja saber ?')

prompt = chat_template.format_messages(regiao=regiao)

print(prompt)

#response = model.invoke(prompt)
#print(response.content)

Vamos deixar agora o chat mais interativo ! 

In [ ]:
import os
from decouple import config
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI

# Configuração da API Key
os.environ['OPENAI_API_KEY'] = config('OPENAI_API_KEY')

# Configurar o modelo
model = ChatOpenAI(
    model_name='gpt-3.5-turbo',
)

# Mensagens iniciais para o contexto do chat
messages = [
    SystemMessage(content='Você é um assistente que fornece informações sobre figuras históricas.')
]

print("Digite 'sair' para encerrar o chat.\n")

# Loop do chat
while True:
    # Entrada do usuário
    question = input('Você: ')

    # Verificar se o usuário quer encerrar o chat
    if question.lower() == 'sair':
        print("Chat encerrado. Até a próxima!")
        break

    # Adicionar a mensagem do usuário à lista de mensagens
    messages.append(HumanMessage(content=question))

    # Obter resposta do modelo
    response = model(messages)

    # Exibir resposta
    print("Assistente:", response.content)

    # Adicionar a resposta da IA às mensagens para manter o contexto
    messages.append(AIMessage(content=response.content))


Agora vamos falar sobre Agentes, os Agents do LangChain possuem ferramentas (toolskits) que podem aumentar significamente o poder as IAs:

In [ ]:
import os
from decouple import config


from langchain.agents import create_react_agent, AgentExecutor # Função de criação de um agente. 
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit # Todas as ferramentas que tratam o SQL.
from langchain_community.utilities.sql_database import SQLDatabase # Classe de conexão com o banco de dados
from langchain_openai import ChatOpenAI
from langchain import hub # armazenamento de prompts prontos da comunidade.
from langchain.prompts import PromptTemplate

# Configuração da API Key
os.environ['OPENAI_API_KEY'] = config('OPENAI_API_KEY')

# Configurar o modelo
model = ChatOpenAI(
    model_name='gpt-3.5-turbo',
)

# Fazendo a conexão com o banco de dados
db = SQLDatabase.from_uri('sqlite:///pop_database.db')

toolkit = SQLDatabaseToolkit(
    db=db,
    llm=model
) 

system_message = hub.pull('hwchase17/react') # Um prompt pronto de um usuário de langchain que vai ajudar a IA se localizar nas ferramentas.
#print(system_message)

# Criando o Agente:
agent = create_react_agent(
    llm = model,
    tools= toolkit.get_tools(),
    prompt = system_message,
)

# Executando o Agente:
agent_execut = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

prompt = '''
Use as ferramentas necessárias para responder perguntas relacionadas aos procedimentos que estão armazenados no banco de dados,
pergunta {q}
'''

prompt_template = PromptTemplate.from_template(prompt)

question = input('O que deseja saber ?')

output = agent_execut.invoke(
    {
        'input':prompt_template.format(q=question),
    }
)

print(output.get('output'))


Vamos deixar um pouco mais interativo ! 

In [ ]:
import os
from decouple import config


from langchain.agents import create_react_agent, AgentExecutor # Função de criação de um agente. 
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit # Todas as ferramentas que tratam o SQL.
from langchain_community.utilities.sql_database import SQLDatabase # Classe de conexão com o banco de dados
from langchain_openai import ChatOpenAI
from langchain import hub # armazenamento de prompts prontos da comunidade.
from langchain.prompts import PromptTemplate

# Configuração da API Key
os.environ['OPENAI_API_KEY'] = config('OPENAI_API_KEY')

# Configurar o modelo
model = ChatOpenAI(
    model_name='gpt-3.5-turbo',
)

# Fazendo a conexão com o banco de dados
db = SQLDatabase.from_uri('sqlite:///pop_database.db')

toolkit = SQLDatabaseToolkit(
    db=db,
    llm=model
) 

system_message = hub.pull('hwchase17/react') # Um prompt pronto de um usuário de langchain que vai ajudar a IA se localizar nas ferramentas.
#print(system_message)

# Criando o Agente:
agent = create_react_agent(
    llm = model,
    tools= toolkit.get_tools(),
    prompt = system_message,
)

# Executando o Agente:
agent_execut = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

prompt = '''
Use as ferramentas necessárias para responder perguntas relacionadas aos procedimentos que estão armazenados no banco de dados,
pergunta {q}
'''

prompt_template = PromptTemplate.from_template(prompt)

print("Digite 'Sair' para encerrar o chat . \n")

while True:
    question = input('O que deseja saber ?')
    
    if question.lower() == 'sair':
        print("Chat encerrado!")
        break

    output = agent_execut.invoke(
        {
            'input':prompt_template.format(q=question),
        }
    )

    print(output.get('output'))
